In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive/csci567

/content/gdrive/MyDrive/csci567


In [ ]:
!ls
%env DATA_DIR=./data
%env EXP_DIR=./experiments

csci567  csci567.egg-info  data  experiments  README.md  setup.py  venv
env: DATA_DIR=./data
env: EXP_DIR=./experiments


In [ ]:
from csci567.models.two_tower_model import *

In [ ]:
# Init
experiment_name = "two_tower"
print(f"Initializing experiment: {experiment_name}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 10000
val_size = 100
learning_rate = 5e-4

Initializing experiment: two_tower_symmetric_net_large


In [ ]:
# Load in training data
print("Loading in training data")
transactions_df = get_train_data(cutoff_date=None)
purchases_df, customers_index_dict, articles_index_dict = get_customer_purchases(
    transactions_df, after="2020-08-22")
articles_df = pd.read_csv(os.path.join(os.environ["DATA_DIR"], "articles.csv"), dtype={"article_id": str}).fillna("None")
articles_features_dicts, num_features, total_features = get_object_features(articles_df)
article_index_dicts = prepare_object_embed_idxs(articles_df, articles_features_dicts)

print(f"# unique customer: {len(customers_index_dict)}")
print(f"# unique articles: {len(articles_index_dict)}")
print(f"# unique article features: {total_features}")
del transactions_df
del articles_df
del articles_features_dicts

with open(os.path.join(os.environ["EXP_DIR"], f"{experiment_name}_customers_index_dict.pkl"), 'wb') as f:
    pickle.dump(customers_index_dict, f)
with open(os.path.join(os.environ["EXP_DIR"], f"{experiment_name}_articles_index_dicts.pkl"), 'wb') as f:
    pickle.dump(article_index_dicts, f)

training_purchases_df = purchases_df
testing_purchases_df = purchases_df[-val_size:]

Loading in training data
# unique customer: 256355
# unique articles: 29548
# unique article features: 152974


In [ ]:
# Make dataset and dataloader
print("Making purchases dataloader")
training_purchases_dataset = PurchasesDataset(
    training_purchases_df, customers_index_dict, article_index_dicts, device=device)
training_purchases_dataloader = DataLoader(
    training_purchases_dataset, batch_size=64, drop_last=True, shuffle=True)
testing_purchases_dataset = PurchasesDataset(
    testing_purchases_df, customers_index_dict, article_index_dicts, device=device)
testing_purchases_dataloader = DataLoader(
    testing_purchases_dataset, batch_size=val_size, drop_last=True, shuffle=False)

Making purchases dataloader


In [ ]:
# Make model and trainer
print("Making model and trainer")
two_tower_model = TwoTowerModel(
    len(customers_index_dict), num_features, device=device)
count_parameters(two_tower_model)
two_tower_trainer = TwoTowerTrainer(
    two_tower_model, training_purchases_dataloader, testing_purchases_dataloader, epochs, learning_rate, experiment_name)

Making model and trainer
+-----------------------------+------------+
|           Modules           | Parameters |
+-----------------------------+------------+
|  queries_embeddings.weight  |  2050840   |
| objects_embeddings.0.weight |   422168   |
| objects_embeddings.1.weight |   377792   |
| objects_embeddings.2.weight |    1056    |
| objects_embeddings.3.weight |    120     |
| objects_embeddings.4.weight |     80     |
| objects_embeddings.5.weight |     20     |
| objects_embeddings.6.weight |     84     |
|   query_net.model.0.weight  |    1408    |
|    query_net.model.0.bias   |     32     |
|   query_net.model.2.weight  |    1024    |
|    query_net.model.2.bias   |     32     |
|   query_net.model.4.weight  |    512     |
|    query_net.model.4.bias   |     16     |
|  object_net.model.0.weight  |    1152    |
|   object_net.model.0.bias   |     32     |
|  object_net.model.2.weight  |    1024    |
|   object_net.model.2.bias   |     32     |
|  object_net.model.4.weight  

In [ ]:
# Start training
print("Starting training")
two_tower_trainer.train()